# Kabuto Auto Trader - 完全バックテスト

OHLCVデータから完全なバックテストを実行します。

## バックテストフロー
1. **Step A**: OHLCVデータ取得（Yahoo Finance）
2. **Step B**: データクリーニング
3. **Step C**: テクニカルインジケーター計算
4. **Step D**: エントリー/エグジットシグナル生成
5. **Step E**: バックテスト実行（K線シミュレーション）
6. **Step F**: 詳細レポート生成

## ライブラリインポート

In [ ]:
import sys
sys.path.append('../lib')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

from market_data import MarketDataFetcher
from data_cleaner import DataCleaner
from indicators import TechnicalIndicators
from signal_generator import SignalGenerator
from backtest_engine import BacktestEngine
from backtest_analytics import BacktestAnalyzer

print("ライブラリのインポート完了")

## Step A: OHLCVデータ取得

Yahoo Financeから株価データを取得します。

### 日本株の銘柄コード例
- **7203.T**: トヨタ自動車
- **9984.T**: ソフトバンクグループ
- **6758.T**: ソニーグループ
- **7974.T**: 任天堂
- **6861.T**: キーエンス

In [ ]:
# ===== 設定 =====
TICKER = '7203.T'  # トヨタ自動車
START_DATE = '2024-01-01'
END_DATE = '2024-12-31'
INTERVAL = '1d'  # '1d'=日足, '1h'=1時間足, '5m'=5分足

# データ取得
fetcher = MarketDataFetcher()
df = fetcher.fetch_ohlcv(TICKER, START_DATE, END_DATE, INTERVAL)

print(f"データ取得完了: {len(df)}行")
print(f"期間: {df['timestamp'].min()} - {df['timestamp'].max()}")
df.head()

## Step B: データクリーニング

In [ ]:
cleaner = DataCleaner(df)
df_clean = (
    cleaner
    .remove_anomalies(min_volume=0)
    .remove_duplicates()
    .fill_missing_values(method='ffill', limit=3)
    .get_cleaned_data()
)

print(cleaner.get_cleaning_summary())
df_clean.head()

## Step C: テクニカルインジケーター計算

Kabuto戦略で使用するインジケーター:
- EMA (5, 25, 75)
- RSI (14)
- ATR (14)
- 出来高MA (20)

In [ ]:
ti = TechnicalIndicators(df_clean)
df_indicators = (
    ti
    .add_all_kabuto_indicators(
        ema_fast=5,
        ema_medium=25,
        ema_slow=75,
        rsi_period=14,
        atr_period=14,
        volume_period=20
    )
    .get_data()
)

print(f"インジケーター追加完了")
print(f"カラム数: {len(df_indicators.columns)}")
df_indicators[['timestamp', 'close', 'ema_fast', 'ema_medium', 'ema_slow', 'rsi', 'atr']].tail(10)

## Step D: シグナル生成

### 戦略パラメータ
必要に応じてカスタマイズしてください。

In [ ]:
# 戦略パラメータ（カスタマイズ可能）
strategy_params = {
    'rsi_lower': 50,
    'rsi_upper': 70,
    'volume_multiplier': 1.2,
    'atr_sl_multiplier': 2.0,
    'atr_tp_multiplier': 4.0,
    'min_rr_ratio': 1.5,
    'max_daily_entries': 3,
    'cooldown_minutes': 30
}

# シグナル生成
sg = SignalGenerator(df_indicators, strategy_params)
df_signals = (
    sg
    .generate_entry_signals()
    .apply_risk_filters()
    .get_signals()
)

sg.print_signal_summary()

# エントリーポイント表示
entry_points = sg.get_entry_points()
print(f"\nエントリーポイント数: {len(entry_points)}")
if len(entry_points) > 0:
    entry_points[['timestamp', 'close', 'stop_loss', 'take_profit', 'rsi']].head(10)

## Step E: バックテスト実行

### バックテスト設定

In [ ]:
# バックテストエンジン初期化
engine = BacktestEngine(
    initial_capital=1000000,      # 初期資金: 100万円
    commission_rate=0.001,        # 手数料: 0.1%
    slippage_rate=0.0005,         # スリッページ: 0.05%
    position_size_pct=0.1,        # ポジションサイズ: 資金の10%
    max_daily_loss=50000,         # 日次最大損失: 5万円
    max_consecutive_losses=5      # 最大連続損失: 5回
)

# バックテスト実行
results = engine.run(df_signals, verbose=False)

print("\n" + "="*70)
print("バックテスト完了")
print("="*70)

## Step F: 結果分析

### 基本サマリー

In [ ]:
engine.print_summary(results['summary'])

### 取引履歴

In [ ]:
trades = results['trades']
print(f"総取引数: {len(trades)}")
trades.head(10)

### 詳細分析レポート

In [ ]:
analyzer = BacktestAnalyzer(
    results['trades'],
    results['capital_curve'],
    engine.initial_capital
)

analyzer.print_comprehensive_report()

## グラフ可視化

### 1. 資金曲線

In [ ]:
capital_curve = results['capital_curve']

plt.figure(figsize=(14, 6))
plt.plot(capital_curve['timestamp'], capital_curve['capital'], label='資金残高', linewidth=2)
plt.axhline(y=engine.initial_capital, color='gray', linestyle='--', label='初期資金')
plt.title('資金曲線', fontsize=16)
plt.xlabel('日時')
plt.ylabel('資金（円）')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 2. ドローダウン

In [ ]:
dd_details = analyzer.calculate_drawdown_details()

if dd_details:
    plt.figure(figsize=(14, 6))
    plt.fill_between(
        capital_curve['timestamp'],
        0,
        dd_details['drawdown_series'],
        color='red',
        alpha=0.3,
        label='ドローダウン'
    )
    plt.title('ドローダウン推移', fontsize=16)
    plt.xlabel('日時')
    plt.ylabel('ドローダウン（円）')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

### 3. 損益分布

In [ ]:
if len(trades) > 0:
    plt.figure(figsize=(10, 6))
    plt.hist(trades['pnl'], bins=30, edgecolor='black', alpha=0.7)
    plt.axvline(x=0, color='red', linestyle='--', linewidth=2, label='損益ゼロ')
    plt.title('損益分布', fontsize=16)
    plt.xlabel('損益（円）')
    plt.ylabel('取引数')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

### 4. 月次リターン

In [ ]:
monthly = analyzer.calculate_monthly_returns()

if len(monthly) > 0:
    monthly['year_month'] = monthly['year'].astype(str) + '-' + monthly['month'].astype(str).str.zfill(2)
    
    plt.figure(figsize=(12, 6))
    colors = ['green' if x > 0 else 'red' for x in monthly['pnl']]
    plt.bar(monthly['year_month'], monthly['pnl'], color=colors, alpha=0.7)
    plt.title('月次損益', fontsize=16)
    plt.xlabel('年月')
    plt.ylabel('損益（円）')
    plt.xticks(rotation=45)
    plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()

## パラメータ最適化（オプション）

複数のパラメータセットを試して最適なものを見つけます。

In [ ]:
# パラメータグリッド
param_grid = [
    {'rsi_lower': 45, 'atr_tp_multiplier': 3.0, 'name': 'Conservative'},
    {'rsi_lower': 50, 'atr_tp_multiplier': 4.0, 'name': 'Standard'},
    {'rsi_lower': 55, 'atr_tp_multiplier': 5.0, 'name': 'Aggressive'},
]

optimization_results = []

for params in param_grid:
    print(f"\nテスト中: {params['name']}")
    
    # シグナル生成
    sg_test = SignalGenerator(df_indicators, params)
    df_test = sg_test.generate_entry_signals().apply_risk_filters().get_signals()
    
    # バックテスト
    engine_test = BacktestEngine(initial_capital=1000000)
    results_test = engine_test.run(df_test, verbose=False)
    
    # 結果保存
    optimization_results.append({
        'name': params['name'],
        'params': params,
        'total_return': results_test['summary']['total_return_pct'],
        'profit_factor': results_test['summary']['profit_factor'],
        'win_rate': results_test['summary']['win_rate'],
        'max_dd_pct': results_test['summary']['max_drawdown_pct']
    })

# 結果比較
opt_df = pd.DataFrame(optimization_results)
print("\n=== パラメータ最適化結果 ===")
opt_df

## まとめ

このノートブックでは、OHLCVデータから完全なバックテストを実行しました。

### 次のステップ
1. ✅ パラメータを調整してバックテストを再実行
2. ✅ 複数銘柄でテスト
3. ✅ 異なる時間軸（5分足、1時間足）でテスト
4. ✅ Walk-Forward Analysis（期間を分割してテスト）
5. ✅ 改善が確認できたら実トレードに適用

### 注意事項
- バックテスト結果は過去データに基づいており、将来のパフォーマンスを保証するものではありません
- 過度な最適化（カーブフィッティング）に注意してください
- 必ずフォワードテストで検証してから実トレードに適用してください